# Interactive Rubric Demo for hiMCM

Edit rubric.json and see scoring changes in real-time.

In [ ]:
!pip install -q pandas numpy seaborn matplotlib

In [ ]:
import json
from pathlib import Path

rubric_path = Path('config') / 'rubric.json'
if not rubric_path.exists():
    rubric_path.parent.mkdir(parents=True, exist_ok=True)
    default = {
        'Stadium_Type': {'dome': 100, 'open-air': 50, 'default': 50},
        'Stadium_leed_cert': {'platinum': 100, 'gold': 80, 'silver': 60, 'default': 30},
        'Waste_Management': {'recycling and composting': 90, 'recycling only': 70, 'default': 50},
        'Future_Developments': {'major_green_investments': 100, 'no_known_plans': 30, 'default': 50}
    }
    rubric_path.write_text(json.dumps(default, indent=2))

print('Current rubric:')
print(rubric_path.read_text())

In [ ]:
import pandas as pd
import sys, os
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

from himcm.core import calculate_sustainability_scores

df = pd.DataFrame([
    {'City': 'City A', 'State': 'ST', 'Times_Hosted': 0, 'Last_Hosted_Year': 1990, 'Stadium_Type': 'open-air', 'Stadium_leed_cert': 'Gold', 'Avg_Feb_Temp_F': 45, 'Alltransit_Score': 70, 'Renewable_Energy_Pct': 20, 'Waste_Diversion_Pct': 30, 'Green_Legacy_Projects': 1, 'Carbon_Offset_Programs': 0, 'Waste_Management': 'recycling only', 'Future_Developments': 'no_known_plans'},
    {'City': 'City B', 'State': 'ST', 'Times_Hosted': 2, 'Last_Hosted_Year': 2015, 'Stadium_Type': 'dome', 'Stadium_leed_cert': 'Platinum', 'Avg_Feb_Temp_F': 30, 'Alltransit_Score': 85, 'Renewable_Energy_Pct': 75, 'Waste_Diversion_Pct': 80, 'Green_Legacy_Projects': 5, 'Carbon_Offset_Programs': 3, 'Waste_Management': 'recycling and composting', 'Future_Developments': 'major_green_investments'}
])

df_before = calculate_sustainability_scores(df.copy())
print('BEFORE edit:')
print(df_before[['City', 'Raw_Score', 'Sustainability_Z_Scaled']])

In [ ]:
rubric = json.loads(rubric_path.read_text())
print('Changing gold from', rubric['Stadium_leed_cert'].get('gold'), 'to 40')
rubric['Stadium_leed_cert']['gold'] = 40
rubric_path.write_text(json.dumps(rubric, indent=2))
print('✓ Rubric updated')

In [ ]:
df_after = calculate_sustainability_scores(df.copy())
print('AFTER edit:')
print(df_after[['City', 'Raw_Score', 'Sustainability_Z_Scaled']])
print('\nDifference:')
print('Raw_Score change:', (df_after['Raw_Score'] - df_before['Raw_Score']).values)